# CircleCI Project Analysis

## Setup and Configuration

In [ ]:
# Setup and Configuration
import os
import analysis
import pandas as pd
from IPython.display import display, HTML

# Initialize the analysis environment
analysis.initialize_notebook()

# Configuration - can be set via environment variables or modified directly
filepath = os.getenv("FILEPATH", "../merged.csv")
credit_cost = float(os.getenv("CREDIT_COST", "0.0006"))

# Set num characters in a column (None is unlimited).
pd.set_option("display.max_colwidth", None)

# Show all columns when running `.head()`
pd.set_option("display.max_columns", None)
pd.set_option("display.colheader_justify", "left")

# Load data to get organization name for report header
temp_df = pd.read_csv(filepath, escapechar="\\", na_values=["\\N"], nrows=1)
org_name = temp_df['ORGANIZATION_NAME'].iloc[0] if 'ORGANIZATION_NAME' in temp_df.columns else "Unknown Organization"

print("=" * 60)
print("CircleCI Project Analysis Report")
print("=" * 60)
print(f"Organization: {org_name}")
print(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()


In [ ]:
# Load CSV data into a dataframe
na_values = ["\\N"]
df = pd.read_csv(filepath, escapechar="\\", na_values=na_values)

# Convert datetime columns
df["JOB_RUN_STARTED_AT"] = pd.to_datetime(df["JOB_RUN_STARTED_AT"], format="ISO8601")

# Calculate date range from JOB_RUN_DATE column
if "JOB_RUN_DATE" in df.columns:
    # Use JOB_RUN_DATE for date range calculation
    df["JOB_RUN_DATE"] = pd.to_datetime(df["JOB_RUN_DATE"])
    query_start_date = df["JOB_RUN_DATE"].min()
    query_end_date = df["JOB_RUN_DATE"].max()
else:
    # Fallback to JOB_RUN_STARTED_AT if JOB_RUN_DATE doesn't exist
    query_start_date = df["JOB_RUN_STARTED_AT"].min().normalize()
    query_end_date = df["JOB_RUN_STARTED_AT"].max().normalize()

query_period_days = (query_end_date - query_start_date).days + 1

print(f"  Query Date Range: {query_start_date.strftime('%Y-%m-%d')} to {query_end_date.strftime('%Y-%m-%d')} ({query_period_days} days)")


In [ ]:
# Add custom fields 

# Cost
df["COST"] = (df["TOTAL_CREDITS"] - df["USER_CREDITS"]) * credit_cost

# Remove fields that aren't of interest
columns_to_remove = [
    "ORGANIZATION_ID", 
    "ORGANIZATION_CREATED_DATE",
    "PROJECT_ID",
    "PROJECT_CREATED_DATE",
    "VCS_NAME",
    "VCS_URL",
    "IS_UNREGISTERED_USER",
    "PIPELINE_TRIGGER_SOURCE",
    "MEDIAN_CPU_UTILIZATION_PCT",
    "MAX_CPU_UTILIZATION_PCT",
    "MEDIAN_RAM_UTILIZATION_PCT",
    "MAX_RAM_UTILIZATION_PCT",
]
df = df.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
# Summarise jobs
all_jobs = df

# Calculate summary statistics using the query date range
num_jobs = len(all_jobs)
num_pipelines = all_jobs["PIPELINE_ID"].nunique() if "PIPELINE_ID" in all_jobs.columns else "Unknown"
total_cost = all_jobs["COST"].sum() if "COST" in all_jobs.columns else 0

print(f"All jobs: {num_jobs:,} jobs from {num_pipelines:,} pipelines - {query_period_days} day period ({query_start_date.strftime('%Y-%m-%d')} to {query_end_date.strftime('%Y-%m-%d')}) - total cost ${total_cost:,.0f}")


# Most Expensive Projects Analysis


In [ ]:
# Analyze the most expensive projects across the organization
# Get organization name from the data
org_name_from_data = all_jobs['ORGANIZATION_NAME'].iloc[0] if len(all_jobs) > 0 else "Unknown Organization"

# Use the query date range for analysis period
# This represents the full period that was queried, not just days with activity
analysis_period_days = query_period_days

print(f"🏆 MOST EXPENSIVE PROJECTS ANALYSIS - {org_name_from_data}")
print("=" * (50 + len(org_name_from_data)))
print(f"\n📅 Analysis Period: {query_start_date.strftime('%Y-%m-%d')} to {query_end_date.strftime('%Y-%m-%d')} ({analysis_period_days} days)")

# Group by project and calculate key metrics
project_costs = all_jobs.groupby('PROJECT_NAME').agg({
    'COST': ['sum', 'mean', 'count'],
    'TOTAL_CREDITS': 'sum',
    'JOB_RUN_SECONDS': ['sum', 'mean']
}).round(2)

# Flatten column names
project_costs.columns = [
    'total_cost', 'avg_cost_per_job', 'total_jobs',
    'total_credits',
    'total_runtime_seconds', 'avg_runtime_seconds'
]

# Calculate additional metrics using the query date range
project_costs['jobs_per_day'] = (project_costs['total_jobs'] / analysis_period_days).round(2)
project_costs['cost_per_day'] = (project_costs['total_cost'] / analysis_period_days).round(2)
project_costs['total_runtime_hours'] = (project_costs['total_runtime_seconds'] / 3600).round(1)

# Filter projects with meaningful data (at least 10 jobs)
significant_projects = project_costs[project_costs['total_jobs'] >= 10]

print(f"\n📊 {org_name_from_data} Overview:")
print(f"  Total Projects: {len(project_costs)}")
print(f"  Projects with ≥10 jobs: {len(significant_projects)}")
print(f"  Total Organization Cost: ${project_costs['total_cost'].sum():,.2f}")
print(f"  Total Organization Credits: {project_costs['total_credits'].sum():,.0f}")

if len(significant_projects) > 0:
    print(f"  Most Active Project: {significant_projects['total_jobs'].idxmax()} ({significant_projects['total_jobs'].max():,.0f} jobs)")
    print(f"  Most Expensive Project: {significant_projects['total_cost'].idxmax()} (${significant_projects['total_cost'].max():,.2f})")



## Top 10 Most Expensive Projects


In [ ]:
# Display top 10 most expensive projects as a table
if len(significant_projects) > 0:
    print("💰 TOP 10 MOST EXPENSIVE PROJECTS")
    print("-" * 80)
    
    top_expensive = significant_projects.nlargest(10, 'total_cost').copy()
    
    # Calculate cost share percentage
    total_org_cost = significant_projects['total_cost'].sum()
    top_expensive['cost_share_pct'] = (top_expensive['total_cost'] / total_org_cost * 100).round(1)
    
    # Create display dataframe with formatted columns
    display_df = pd.DataFrame({
        'Project': top_expensive.index,
        'Total Cost': top_expensive['total_cost'].apply(lambda x: f"${x:,.2f}"),
        'Cost Share': top_expensive['cost_share_pct'].apply(lambda x: f"{x:.1f}%"),
        'Total Jobs': top_expensive['total_jobs'].astype(int),
        'Jobs/Day': top_expensive['jobs_per_day'].round(1),
        'Total Credits': top_expensive['total_credits'].astype(int).apply(lambda x: f"{x:,}"),
        'Runtime (hrs)': top_expensive['total_runtime_hours'].round(1),
        'Avg Cost/Job': top_expensive['avg_cost_per_job'].apply(lambda x: f"${x:.3f}")
    })
    
    # Reset index to show row numbers starting from 1
    display_df.index = range(1, len(display_df) + 1)
    
    # Display the table
    display(HTML(display_df.to_html(escape=False)))
    
else:
    print("❌ No projects available for top expensive analysis")



## Cost Efficiency Analysis


In [ ]:
# Analyze cost efficiency and identify optimization opportunities
if len(significant_projects) > 0:
    print(f"⚡ COST EFFICIENCY & OPTIMIZATION OPPORTUNITIES - {org_name_from_data}")
    print("=" * (55 + len(org_name_from_data)))
    
    # Calculate efficiency metrics
    significant_projects['runtime_per_job_minutes'] = significant_projects['avg_runtime_seconds'] / 60
    
    # Find most and least efficient projects
    most_efficient = significant_projects.nsmallest(5, 'avg_cost_per_job')
    least_efficient = significant_projects.nlargest(5, 'avg_cost_per_job')
    
    # Display Most Cost-Efficient Projects Table
    print(f"\n🏆 MOST COST-EFFICIENT PROJECTS (Lowest cost per job):")
    print("-" * 80)
    efficient_df = pd.DataFrame({
        'Project': most_efficient.index,
        'Avg Cost/Job': most_efficient['avg_cost_per_job'].apply(lambda x: f"${x:.3f}"),
        'Total Cost': most_efficient['total_cost'].apply(lambda x: f"${x:.2f}"),
        'Total Jobs': most_efficient['total_jobs'].astype(int),
        'Avg Runtime (min)': most_efficient['runtime_per_job_minutes'].round(1),
        'Jobs/Day': most_efficient['jobs_per_day'].round(1)
    })
    efficient_df.index = range(1, len(efficient_df) + 1)
    display(HTML(efficient_df.to_html(escape=False)))
    
    # Display Least Cost-Efficient Projects Table
    print(f"\n⚠️  LEAST COST-EFFICIENT PROJECTS (Highest cost per job):")
    print("-" * 80)
    inefficient_df = pd.DataFrame({
        'Project': least_efficient.index,
        'Avg Cost/Job': least_efficient['avg_cost_per_job'].apply(lambda x: f"${x:.3f}"),
        'Total Cost': least_efficient['total_cost'].apply(lambda x: f"${x:.2f}"),
        'Total Jobs': least_efficient['total_jobs'].astype(int),
        'Avg Runtime (min)': least_efficient['runtime_per_job_minutes'].round(1),
        'Jobs/Day': least_efficient['jobs_per_day'].round(1)
    })
    inefficient_df.index = range(1, len(inefficient_df) + 1)
    display(HTML(inefficient_df.to_html(escape=False)))
    
    # Activity vs Cost analysis
    high_activity = significant_projects[significant_projects['jobs_per_day'] > significant_projects['jobs_per_day'].quantile(0.75)]
    
    if len(high_activity) > 0:
        print(f"\n🚀 HIGH-ACTIVITY PROJECTS (Top 25% by jobs/day):")
        print("   (Focus areas for optimization due to high frequency)")
        print("-" * 80)
        
        activity_df = pd.DataFrame({
            'Project': high_activity.nlargest(5, 'jobs_per_day').index,
            'Jobs/Day': high_activity.nlargest(5, 'jobs_per_day')['jobs_per_day'].round(1),
            'Cost/Day': high_activity.nlargest(5, 'jobs_per_day')['cost_per_day'].apply(lambda x: f"${x:.2f}"),
            'Total Jobs': high_activity.nlargest(5, 'jobs_per_day')['total_jobs'].astype(int),
            'Total Cost': high_activity.nlargest(5, 'jobs_per_day')['total_cost'].apply(lambda x: f"${x:.2f}"),
            'Avg Cost/Job': high_activity.nlargest(5, 'jobs_per_day')['avg_cost_per_job'].apply(lambda x: f"${x:.3f}")
        })
        activity_df.index = range(1, len(activity_df) + 1)
        display(HTML(activity_df.to_html(escape=False)))
    
    # Summary recommendations
    total_org_cost = significant_projects['total_cost'].sum()
    top_3_cost = significant_projects.nlargest(3, 'total_cost')['total_cost'].sum()
    top_3_share = (top_3_cost / total_org_cost * 100) if total_org_cost > 0 else 0
    
    print(f"\n💡 OPTIMIZATION RECOMMENDATIONS:")
    print(f"   1. 🎯 Focus on top 3 projects (${top_3_cost:,.2f}, {top_3_share:.1f}% of total cost)")
    print(f"   2. ⚡ Review resource allocation for high-activity projects")
    print(f"   3. 🔍 Investigate projects with >$0.010 cost per job")
    print(f"   4. 📊 Set up monitoring for projects spending >${significant_projects['cost_per_day'].quantile(0.9):.2f}/day")
    print(f"   5. 🏃 Consider build optimization for projects with >{significant_projects['runtime_per_job_minutes'].quantile(0.9):.0f} min avg runtime")
    
else:
    print("❌ No projects available for efficiency analysis")

